# Data Cleaning and Preparation 数据清洗和准备

In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data 处理缺失值

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data  过滤缺失值

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()]

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data
cleaned

In [ ]:
data.dropna(how='all')

In [ ]:
data[4] = NA
data
data.dropna(axis=1, how='all')

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2)

### Filling In Missing Data  填充缺失值

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
_ = df.fillna(0, inplace=True)
df

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

## Data Transformation

### Removing Duplicates

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping  用函数或者映射转换数据

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### Replacing Values   替换值

In [5]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [6]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [7]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [8]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [9]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes  重命名轴索引

In [4]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [9]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [10]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [15]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Indiana,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [14]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [13]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning 离散化和装箱

In [22]:
ages = [18, 20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [23]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)  #默认right = True 不包含左边，包含右边
cats

[NaN, (18.0, 25.0], (18.0, 25.0], (18.0, 25.0], (25.0, 35.0], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 13
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [18]:
print(cats.codes)
print(cats.categories)
pd.value_counts(cats)

[0 0 0 1 0 0 2 1 3 2 2 1]
IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')


(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [21]:
# [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages = [17, 20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
pd.cut(ages, [18, 26, 36, 61, 100], right=False)  #默认 不包含右边，包含左边

[NaN, [18.0, 26.0), [18.0, 26.0), [18.0, 26.0), [26.0, 36.0), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 13
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [34]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
lbs = pd.cut(ages, bins, labels=group_names)
# lbs.categories[0]
lbs
# list(lbs)

[NaN, 'Youth', 'Youth', 'Youth', 'YoungAdult', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 13
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [35]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.73, 0.96], (0.25, 0.49], (0.0074, 0.25], (0.0074, 0.25], (0.49, 0.73], ..., (0.49, 0.73], (0.73, 0.96], (0.73, 0.96], (0.73, 0.96], (0.49, 0.73]]
Length: 20
Categories (4, interval[float64]): [(0.0074, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.96]]

In [37]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
print(cats)
pd.value_counts(cats)

[(-0.0189, 0.681], (-3.746, -0.623], (-0.0189, 0.681], (0.681, 3.26], (-0.623, -0.0189], ..., (0.681, 3.26], (-3.746, -0.623], (-0.0189, 0.681], (0.681, 3.26], (-3.746, -0.623]]
Length: 1000
Categories (4, interval[float64]): [(-3.746, -0.623] < (-0.623, -0.0189] < (-0.0189, 0.681] < (0.681, 3.26]]


(0.681, 3.26]        250
(-0.0189, 0.681]     250
(-0.623, -0.0189]    250
(-3.746, -0.623]     250
dtype: int64

In [39]:
cats2 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
pd.value_counts(cats2)

(-0.0189, 1.338]     400
(-1.296, -0.0189]    400
(1.338, 3.26]        100
(-3.746, -1.296]     100
dtype: int64

### Detecting and Filtering Outliers 检测和过滤异常值

In [40]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.035864,-0.034753,-0.032879,0.024807
std,0.992153,0.959918,0.988937,1.011569
min,-3.333767,-2.901831,-3.428254,-3.645860
25%,-0.617533,-0.701202,-0.728577,-0.659710
50%,0.018743,-0.050251,-0.064342,0.047018
75%,0.688640,0.664260,0.624413,0.723090
max,3.525865,2.611678,3.366626,2.763474


In [46]:
col = data[2]
# print(col)
col[np.abs(col) > 3]

19    -3.428254
396    3.366626
892   -3.108915
Name: 2, dtype: float64

In [47]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
19,0.326045,0.425384,-3.428254,-0.296336
83,-3.184377,1.369891,-1.074833,-0.089937
304,0.208011,-0.150923,-0.362528,-3.548824
396,0.193299,1.397822,3.366626,-2.372214
543,3.525865,0.283070,0.544635,0.462204
563,-0.450721,-0.080332,0.599947,-3.645860
786,-3.333767,-1.240685,-0.650855,0.076254
867,0.344072,0.581893,-1.116332,-3.018842
892,-0.555434,-0.048478,-3.108915,1.117755


In [48]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.035857,-0.034753,-0.032709,0.026021
std,0.988774,0.959918,0.986027,1.007572
min,-3.000000,-2.901831,-3.000000,-3.000000
25%,-0.617533,-0.701202,-0.728577,-0.659710
50%,0.018743,-0.050251,-0.064342,0.047018
75%,0.688640,0.664260,0.624413,0.723090
max,3.000000,2.611678,3.000000,2.763474


In [49]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,-1.0,1.0,1.0,1.0
2,-1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,1.0
4,-1.0,1.0,-1.0,-1.0


### Permutation and Random Sampling 排列和随机取样

In [50]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([4, 0, 3, 1, 2])

In [ ]:
df
df.take(sampler)

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

In [51]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [52]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [4]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

d:\python38\lib\site-packages\pandas\io\parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [5]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
# print(genres)

In [6]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [10]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [11]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [12]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [14]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic[:5]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
np.random.seed(12345)
values = np.random.rand(10)
print(values)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

[0.9296 0.3164 0.1839 0.2046 0.5677 0.5955 0.9645 0.6532 0.7489 0.6536]


,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


In [10]:
r = np.random.RandomState(12345)
values = r.rand(10)
print(values)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

[0.9296 0.3164 0.1839 0.2046 0.5677 0.5955 0.9645 0.6532 0.7489 0.6536]


,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

### String Object Methods

In [11]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [12]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [13]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [14]:
'::'.join(pieces)

'a::b::guido'

In [15]:
print('guido' in val)
print(val.index(','))
print(val.find(':'))

True
1
-1


In [16]:
val.index(':')

ValueError: substring not found

In [17]:
val.count(',')

2

In [18]:
print(val.replace(',', '::'))
val.replace(',', '')

a::b::  guido


'ab  guido'

### Regular Expressions

In [20]:
import re
text = "foo    bar\t baz  \tqux"
print(text)
re.split('\s+', text)

foo    bar	 baz  	qux


['foo', 'bar', 'baz', 'qux']

In [21]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [22]:
regex.findall(text)

['    ', '\t ', '  \t']

In [23]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [24]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [26]:
m = regex.search(text)
print(m)
text[m.start():m.end()]

<re.Match object; span=(5, 20), match='dave@google.com'>


'dave@google.com'

In [33]:
print(regex.match(text))

None


In [34]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

### Vectorized String Functions in pandas

In [27]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [30]:
data.str.contains('gmail')

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [29]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [dave@google.com]
Steve    [steve@gmail.com]
Rob        [rob@gmail.com]
Wes                    NaN
dtype: object

In [31]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [39]:
matches = data.str.findall(pattern, flags=re.IGNORECASE)
print(matches.str.get(1))
matches.str[0], matches.str[1]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64


(Dave     dave@google.com
 Steve    steve@gmail.com
 Rob        rob@gmail.com
 Wes                  NaN
 dtype: object,
 Dave    NaN
 Steve   NaN
 Rob     NaN
 Wes     NaN
 dtype: float64)

In [40]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [41]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion